# FINAL PROJECT TLCN - CHATBOT HỖ TRỢ TRA CỨU THÔNG TIN SỨC KHỎE
- Author:
  - Đặng Kim Thành - 21110298
  - Bùi Quốc Khang - 21110202
## Phần 2 - MeDiChat ViVi - Fine-tunning Chatbot

### I. Xử lý dữ liệu

#### Import môi trường và thư viện

In [1]:
import torch

# Check what version of PyTorch is installed
print(torch.__version__)

# Check the current CUDA version being used
print("CUDA Version: ", torch.version.cuda)

# Check if CUDA is available and if so, print the device name
print("Device name:", torch.cuda.get_device_properties("cuda").name)

# Check if FlashAttention is available
print("FlashAttention available:", torch.backends.cuda.flash_sdp_enabled())


2.3.1+cu121
CUDA Version:  12.1
Device name: NVIDIA GeForce RTX 4060 Ti
FlashAttention available: True


In [2]:
import torch
from xformers.ops.fmha.torch_attention_compat import is_pt_flash_compatible
print(torch.backends.cuda.flash_sdp_enabled())
print(is_pt_flash_compatible(force=False))

True
False


In [1]:
!pip list

Package                            Version
---------------------------------- --------------------
absl-py                            1.4.0
accelerate                         0.34.2
aiohappyeyeballs                   2.4.3
aiohttp                            3.10.10
aiosignal                          1.3.1
alabaster                          0.7.16
albucore                           0.0.16
albumentations                     1.4.15
altair                             4.2.2
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.5.1
arviz                              0.19.0
astropy                            6.1.4
astropy-iers-data                  0.2024.10.21.0.33.21
astunparse                         1.6.3
async-timeout                      4.0.3
atpublic                           4.1.0
attrs                              24.2.0
audioread      

In [ ]:
!pip install -q accelerate==1.0.1 peft==0.13.2 bitsandbytes==0.44.1 transformers==4.44.2 trl==0.11.4 langchain==0.3.4 langchain_community==0.3.3 langchain_huggingface==0.1.0 langchain-pinecone==0.2.0 pinecone==5.3.1
# !pip install "unsloth[cu121-torch220] @ git+https://github.com/unslothai/unsloth.git" # For Thanh's local
!pip install "unsloth[cu118-ampere-torch240] @ git+https://github.com/unslothai/unsloth.git"  # For Bui's local

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### Kiểm tra phiên bản cần cài của unsloth

In [ ]:
try: import torch
except: raise ImportError("Install torch via `pip install torch`")
from packaging.version import Version as V
v = V(torch.__version__)
cuda = str(torch.version.cuda)
is_ampere = torch.cuda.get_device_capability()[0] >= 8
if cuda != "12.1" and cuda != "11.8": raise RuntimeError(f"CUDA = {cuda} not supported!")
if   v <= V('2.1.0'): raise RuntimeError(f"Torch = {v} too old!")
elif v <= V('2.1.1'): x = 'cu{}{}-torch211'
elif v <= V('2.1.2'): x = 'cu{}{}-torch212'
elif v  < V('2.3.0'): x = 'cu{}{}-torch220'
elif v  < V('2.4.0'): x = 'cu{}{}-torch230'
elif v  < V('2.5.0'): x = 'cu{}{}-torch240'
else: raise RuntimeError(f"Torch = {v} too new!")
x = x.format(cuda.replace(".", ""), "-ampere" if is_ampere else "")
print(f'pip install --upgrade pip && pip install "unsloth[{x}] @ git+https://github.com/unslothai/unsloth.git"')

#### Thư viện xử lý data

In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
import json

#### Thư viện xử lý finetune

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging, DataCollatorForSeq2Seq
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import standardize_sharegpt
from unsloth.chat_templates import train_on_responses_only
from transformers import TextStreamer

c:\Python\mlperf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: If you want to finetune Gemma 2, upgrade flash-attn to version 2.6.3 or higher!
Newer versions support faster and less memory usage kernels for Gemma 2's attention softcapping!
To update flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
import os

# Lấy và in ra đường dẫn thư mục hiện tại
current_path = os.getcwd()
print("Path hiện tại là:", current_path)


Path hiện tại là: c:\Python\mlperf_env


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

#### Khai báo hàm xử lý prompt dữ liệu đầu vào

- Prompt Vietnamese

In [4]:
def convert_to_prompt_format(disease_data):
  messages = []

  for disease in disease_data["diseases"]:
    l_messages = []
    system_message = {
        "from": "system",
        "value": "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể, đồng thời giữ an toàn. Nếu một câu hỏi không có ý nghĩa hoặc không hợp lý về mặt thông tin, hãy giải thích tại sao thay vì trả lời một điều gì đó không chính xác, vui lòng không chia sẻ thông tin sai lệch."
    }

    user_message = {
        "from": "human",
        "value": f"Mô tả về bệnh {disease['name']}?"
    }
    assistant_message = {
        "from": "gpt",
        "value": f"Mô tả: {disease['description']}.\n\nTriệu chứng: {', '.join(disease['symptoms'])}."
    }
    l_messages.append(system_message)
    l_messages.append(user_message)
    l_messages.append(assistant_message)
    messages.append(l_messages)

    l_messages = []
    user_message = {
        "from": "human",
        "value": f"Triệu chứng về bệnh {disease['name']}?"
    }
    assistant_message = {
        "from": "gpt",
        "value": f"Triệu chứng: {', '.join(disease['symptoms'])}.\n\nPhương pháp điều trị: {', '.join(disease['treatments'])}."
    }
    l_messages.append(system_message)
    l_messages.append(user_message)
    l_messages.append(assistant_message)
    messages.append(l_messages)

    l_messages = []
    user_message = {
        "from": "human",
        "value": f"Phương pháp chữa bệnh {disease['name']}?"
    }
    assistant_message = {
        "from": "gpt",
        "value": f"Phương pháp điều trị: {', '.join(disease['treatments'])}."
    }
    l_messages.append(system_message)
    l_messages.append(user_message)
    l_messages.append(assistant_message)
    messages.append(l_messages)

  return messages

- Prompt English

In [ ]:
def convert_to_prompt_format(disease_data):
    messages = []

    for disease in disease_data["diseases"]:
        l_messages = []
        system_message = {
            "from": "system",
            "value": "You are a helpful and honest assistant. Always provide the most useful answers possible, while ensuring safety. If a question is meaningless or unreasonable, explain why instead of providing incorrect information, and please do not share misleading information."
        }
        user_message = {
            "from": "human",
            "value": f"Describe the disease {disease['name']}?"
        }
        assistant_message = {
            "from": "gpt",
            "value": f"Description: {disease['description']}.\n\nSymptoms: {', '.join(disease['symptoms'])}."
        }
        l_messages.append(system_message)
        l_messages.append(user_message)
        l_messages.append(assistant_message)
        messages.append(l_messages)

        l_messages = []
        l_messages.append(system_message)
        user_message = {
            "from": "human",
            "value": f"What are the symptoms of the disease {disease['name']}?"
        }
        assistant_message = {
            "from": "gpt",
            "value": f"Symptoms: {', '.join(disease['symptoms'])}.\n\nTreatment methods: {', '.join(disease['treatments'])}."
        }
        l_messages.append(user_message)
        l_messages.append(assistant_message)
        messages.append(l_messages)

        l_messages = []
        l_messages.append(system_message)
        user_message = {
            "from": "human",
            "value": f"What are the treatment methods for the disease {disease['name']}?"
        }
        assistant_message = {
            "from": "gpt",
            "value": f"Treatment methods: {', '.join(disease['treatments'])}."
        }
        l_messages.append(user_message)
        l_messages.append(assistant_message)
        messages.append(l_messages)

    return messages

In [5]:
def read_json_and_convert_to_txt(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        disease_data = json.load(infile)

    messages = convert_to_prompt_format(disease_data)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write(json.dumps(messages, ensure_ascii=False, indent=2))

#### Hàm lưu file

In [6]:
input_file = current_path+'\data\health-data-vi.json'
output_file = current_path+'\data\health-data-vi.txt'
read_json_and_convert_to_txt(input_file, output_file)

#### Dataset new

In [4]:
ds = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")
for split, dataset in ds.items():
    dataset.to_csv(f"dataset-mec100k{split}.csv", index = None)

Creating CSV from Arrow format: 100%|██████████| 113/113 [00:02<00:00, 50.69ba/s]


In [9]:
import pandas as pd
import json

file_path = current_path + '/data/dataset-mec100ktrain.csv'  
df = pd.read_csv(file_path, usecols=['human', 'gpt'])

system_message = {
    "from": "system",
    "value": "You are a dedicated and honest English assistant. Always respond as helpfully as possible while ensuring safety. If a question is nonsensical or lacks logical information, explain why instead of providing an inaccurate response. Please avoid sharing misinformation."
}

output_data = []

for _, row in df.iterrows():
    human_message = {"from": "human", "value": str(row['human'])}  
    gpt_message = {"from": "gpt", "value": str(row['gpt'])}     
    conversation = [system_message, human_message, gpt_message]
    output_data.append(conversation)

formatted_output = json.dumps(output_data, ensure_ascii=False, indent=2)

output_file_path = current_path + '/data/health-data-en.txt' 
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(formatted_output)

print("File generated at:", output_file_path)


File generated at: c:\Python\mlperf_env/data/health-data-en.txt


#### Chuẩn hóa dữ liệu thành Document của LangChain

In [4]:
class UTF8TextLoader(TextLoader):
    def __init__(self, file_path):
        super().__init__(file_path, encoding='utf-8')

pdf_path = current_path+'\data'
loader = DirectoryLoader(path=pdf_path, glob='*.txt', loader_cls=UTF8TextLoader)
documents = loader.load()

In [ ]:
documents

### II. Fine-tune model

#### Format file huấn luyện theo định dạng LLama3

In [5]:
all_conversations = []
for doc in documents:
    conversations = json.loads(doc.page_content)
    for conversation in conversations:
        formatted_conversation = [
            {k: str(v) for k, v in message.items()}
            for message in conversation
        ]
        all_conversations.append(formatted_conversation)

data_dict = {"conversations": all_conversations}
dataset = Dataset.from_dict(data_dict)

print("Dataset created successfully.")

Dataset created successfully.


In [5]:
all_conversations = []
for doc in documents:
    conversations = json.loads(doc.page_content)
    all_conversations.extend(conversations)

data_dict = {"conversations": all_conversations}
dataset = Dataset.from_dict(data_dict)

In [12]:
dataset[5]['conversations']

[{'from': 'system',
  'value': 'You are a dedicated and honest English assistant. Always respond as helpfully as possible while ensuring safety. If a question is nonsensical or lacks logical information, explain why instead of providing an inaccurate response. Please avoid sharing misinformation.'},
 {'from': 'human',
  'value': 'I am F 38 in good shape work out (do triathlons) regular but have had back pain from different reasons throughout my life now I every so often wake up with severe lower back, hip pain for no reason. today the pain is almost taking my breathe away when i move. It is a dull pain when i am just lying down but the moment I make any sort of movement I have sharp and sometimes shooting pain down my legs.'},
 {'from': 'gpt',
  'value': 'Hi, From history it seems that you might be having degenerative changes in your lower back spines giving pinched nerve pressure. There might be having osteomalacia or osteoporosis as well. Go for x-ray lumbosacral region for osteoarth

#### Quantize model and set up hyperparameters

In [6]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "model\\Llama-3.2-3B-Instruct-Frog",
    max_seq_length = max_seq_length,
    dtype = torch.bfloat16  ,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4060 Ti. Max memory: 15.995 GB. Platform = Windows.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


c:\Python\mlperf_env\lib\site-packages\unsloth\models\llama.py:1107: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.61s/it]
model\Llama-3.2-3B-Instruct-Frog does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


- Vì model không có padding token nên hệ thống sẽ tự động sử dụng một token mặc định là <|finetune_right_pad_id|> làm padding token. Token này chỉ đơn thuần được sử dụng để làm đầy chuỗi và không mang ý nghĩa nội dung.

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.1, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.10.7 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


#### Thiết lập tokenizer và chuẩn hóa dữ liệu

In [8]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [9]:
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 138643/138643 [00:07<00:00, 18081.81 examples/s]


In [10]:
dataset[5]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a dedicated and honest English assistant. Always respond as helpfully as possible while ensuring safety. If a question is nonsensical or lacks logical information, explain why instead of providing an inaccurate response. Please avoid sharing misinformation.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nI am F 38 in good shape work out (do triathlons) regular but have had back pain from different reasons throughout my life now I every so often wake up with severe lower back, hip pain for no reason. today the pain is almost taking my breathe away when i move. It is a dull pain when i am just lying down but the moment I make any sort of movement I have sharp and sometimes shooting pain down my legs.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHi, From history it seems that you might be having degenerative changes in your lower back

#### Set up file dataset and SFT training

In [11]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import json

conversations = dataset["conversations"]

train_conversations, eval_conversations = train_test_split(conversations, test_size=0.2, random_state=3407)

dataset_dict = DatasetDict({
    "train": Dataset.from_dict({"conversations": train_conversations}),
    "eval": Dataset.from_dict({"conversations": eval_conversations})
})

eval_dataset = dataset_dict["eval"]
train_dataset = dataset_dict["train"]


In [12]:
eval_dataset[5]

{'conversations': [{'content': 'You are a dedicated and honest English assistant. Always respond as helpfully as possible while ensuring safety. If a question is nonsensical or lacks logical information, explain why instead of providing an inaccurate response. Please avoid sharing misinformation.',
   'role': 'system'},
  {'content': 'Hi my daughter aged 5 yrs...she little autistic features and hyper actrivity.so giving axepta 18 at bed time . then she not yet start speaking like other kids. giving speach theraphy last 8 mnths. not yet speak. and taking tablet oxetol 300 ...2 times for epilepsy . last 2yrs. with this strocit drops, nutropil syrup also giving. now in eeg record she normal mns no epilepsy features. did she improve from this situation and moove like other kids. Smitha from Kerala.',
   'role': 'user'},
  {'content': "Hi.for epilepsy, the medication is already chosen and for hyperactivity also.but autism is a kind of neurodevelopmental disorder for which medication do not 

In [13]:
train_dataset[5]

{'conversations': [{'content': 'You are a dedicated and honest English assistant. Always respond as helpfully as possible while ensuring safety. If a question is nonsensical or lacks logical information, explain why instead of providing an inaccurate response. Please avoid sharing misinformation.',
   'role': 'system'},
  {'content': 'What is the definition of mucositis and what treatments are currently available for it?',
   'role': 'user'},
  {'content': 'Treatment of mucositis is mainly supportive. Oral hygiene is the mainstay of treatment; patients are encouraged to clean their mouth every four hours and at bedtime, more often if the mucositis becomes worse. Water-soluble jellies can be used to lubricate the mouth. Salt mouthwash can soothe the pain and keep food particles clear so as to avoid infection. Patients are also encouraged to drink plenty of liquids, at least three liters a day, and avoid alcohol. Citrus fruits, alcohol, and foods that are hot are all known to aggravate m

In [10]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,
    packing = True, 
    args = TrainingArguments(
        per_device_train_batch_size = 4, 
        gradient_accumulation_steps = 4, 
        warmup_steps = 5, 
        num_train_epochs = 2, 
        max_steps = -1,
        learning_rate = 2e-4, 
        bf16 = is_bfloat16_supported(), 
        max_grad_norm = 0.3, 
        logging_steps = 10,
        optim = "adamw_8bit", 
        save_total_limit=2,
        weight_decay = 0.01, 
        lr_scheduler_type = "linear", 
        seed = 3407,
        save_steps=100,
        output_dir = "/Models/outputs",
        report_to = "none", 
    ),
)

Generating train split: 21415 examples [00:42, 498.71 examples/s]


In [11]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map: 100%|██████████| 21415/21415 [00:30<00:00, 708.05 examples/s]


In [12]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

" area. It is painful to the touch. He keeps insisting it isn t his kidney because the pain is not the same. He is a stubborn man...any advice?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHelloThanks for query. You have not specified about which kidney is functioning and which side stent was placed in 13 years ago. The pain in flank is mostly due to HY Chat Doctor. 1) Urine routine and culture.2) Ultrasound scanning of abdomen and pelvis3) DTPA scan of the kidneys.4) Intravenous Paleography to assess the kidney function on both sides. Further treatment will depend upon result of these tests and final diagnosis<|eot_id|><|eot_id|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a dedicated and honest English assistant. Always respond as helpfully as possible while ensuring safety. If a question is nonsensical or lacks logical information, explain why instead of providing an inaccurate 

In [13]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                      \n\nHelloThanks for query. You have not specified about which kidney is functioning and which side stent was placed in 13 years ago. The pain in flank is mostly due to HY Chat Doctor. 1) Urine routine and culture.2) Ultrasound scanning of abdomen and pelvis3) DTPA scan of the kidneys.4) Intravenous Paleography to assess the kidney function on both sides. Further treatment will depend upon result of these tests and final diagnosis<|eot_id|><|eot_id|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a dedicated and honest English assistant. Always respond as helpfully as possible while ensuring safety. If a question is nonsensical or lacks logical information, explain why instead of providing an inaccurate response. Please avoid sharing misinformation.<|eot_id|>                                                                                                   \n\n

In [14]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060 Ti. Max memory = 15.995 GB.
2.762 GB of memory reserved.


#### Train model

In [15]:
trainer_stats = trainer.train() 

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 21,415 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 2,676
 "-____-"     Number of trainable parameters = 12,156,928
  0%|          | 10/2676 [04:31<20:05:23, 27.13s/it]

{'loss': 2.0244, 'grad_norm': 0.38352692127227783, 'learning_rate': 0.00019962560838637216, 'epoch': 0.01}


  1%|          | 20/2676 [09:03<20:04:10, 27.20s/it]

{'loss': 1.8207, 'grad_norm': 0.25742506980895996, 'learning_rate': 0.00019887682515911645, 'epoch': 0.01}


  1%|          | 30/2676 [13:34<19:57:08, 27.15s/it]

...

{'loss': 1.1719, 'grad_norm': 0.3836185932159424, 'learning_rate': 1.1980531636091353e-06, 'epoch': 1.99}


100%|█████████▉| 2670/2676 [19:54:46<02:41, 26.91s/it]

{'loss': 1.1925, 'grad_norm': 0.3586898446083069, 'learning_rate': 4.492699363534257e-07, 'epoch': 1.99}


100%|██████████| 2676/2676 [19:57:28<00:00, 26.94s/it]c:\Python\mlperf_env\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in model\Llama-3.2-3B-Instruct-Frog - will assume that the vocabulary was not modified.
  warnings.warn(
100%|██████████| 2676/2676 [19:57:28<00:00, 26.85s/it]

{'train_runtime': 71848.7955, 'train_samples_per_second': 0.596, 'train_steps_per_second': 0.037, 'train_loss': 1.2664818635435917, 'epoch': 2.0}


#### Áp dụng khả năng suy luận của mô hình

In [28]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) 

messages = [
    {"role": "user", "content": "Làm sao để chữa bệnh sốt xuất huyết?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, 
    return_tensors = "pt",
    padding=True,  
).to("cuda")

In [30]:
attention_mask = (inputs != tokenizer.pad_token_id).long()  # Đánh dấu tất cả các vị trí khác với pad token id

outputs = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask,  # Sử dụng attention mask
    max_new_tokens=64,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

In [31]:
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nLàm sao để chữa bệnh sốt xuất huyết?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nPhương pháp điều trị: Fluid replacement, Pain relievers, Hospitalization (in severe cases).<|eot_id|>']

#### Sử dụng Text Streamer cho sinh câu trả lời

In [32]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Đau đầu là dấu hiệu của các bệnh nào?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    padding = True,  # Thêm padding để đảm bảo chuỗi có cùng độ dài
).to("cuda")

attention_mask = (inputs != tokenizer.pad_token_id).long()  # Đánh dấu tất cả các vị trí khác với pad token id

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask,  # Sử dụng attention mask
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

Đau đầu là dấu hiệu của các bệnh: Migraines, Headaches due to high blood pressure, Migraines, Migraines with aura, Migraines with visual distortions, Migraines with speech impairment, Migraines with numbness or tingling, Migranes with pain on side of the head, Migranes with confusion, Migranes with blurring of vision, Migranes with dizziness, Migranes with nausea or vomiting, Migranes with light sensitivity, Migranes with sensitivity to loud or sudden noises, Migranes with exhaustion, Migranes with irritable


<a name="Save"></a>
#### Lưu và tải mô hình đã tinh chỉnh
Để lưu mô hình cuối cùng dưới dạng LoRA adapters, bạn có thể sử dụng `push_to_hub` của Huggingface để lưu trực tuyến hoặc `save_pretrained` để lưu tại chỗ (local).

**[NOTE]** Điều này chỉ lưu các LoRA adapters, không phải toàn bộ mô hình. Để lưu dưới dạng 16bit hoặc GGUF, hãy cuộn xuống dưới!

In [25]:
model.save_pretrained("Models/HCM_4BIT00008")
tokenizer.save_pretrained("Models/HCM_4BIT00008")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

c:\Python\mlperf_env\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in model\Llama-3.2-3B-Instruct-Frog - will assume that the vocabulary was not modified.
  warnings.warn(


('Models/HCM_4BIT00007\\tokenizer_config.json',
 'Models/HCM_4BIT00007\\special_tokens_map.json',
 'Models/HCM_4BIT00007\\tokenizer.json')

- Vì chỉ lưu các tham số LoRA nên khi muốn sử dụng model này phải load model base và kết hợp với các tham số loRA để sử dụng

Nếu bạn muốn tải các LoRA adapters mà chúng ta vừa lưu để suy luận, hãy đặt giá trị `False` thành `True`:

In [3]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
if True:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "Models/4BIT00006", 
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        trust_remote_code=True
    )
    FastLanguageModel.for_inference(model) 

messages = [
    {"role": "user", "content": "I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
    padding = True, 
).to("cuda")

attention_mask = (inputs != tokenizer.pad_token_id).long()  

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask, 
    streamer=text_streamer,
    max_new_tokens=256,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4060 Ti. Max memory: 15.995 GB. Platform = Windows.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


c:\Python\mlperf_env\lib\site-packages\unsloth\models\llama.py:1107: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)
Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.92s/it]
Models/4BIT00006 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


It's great that you described your symptoms and their triggers so clearly! I'd like to help you with the question you've submitted about normal stomach discomfort during a spinning sensation. 

When the entire world seems to be spinning around, this is known as vertigo. In many cases, the cause of vertigo isn't fully understood, but it is sometimes due to inner ear problems. These problems involve the balance center and affect your body's equilibrium, which helps it keep its balance while performing daily activities. 

Since your symptoms are caused by inner ear problems and not other illnesses, stomach discomfort during vertigo is a normal and expected part of the condition. It is very important to be aware that discomfort in the stomach can lead to issues such as gastritis or gastric ulcers, so it should be looked at if it lasts a long time or gets worse over time. If the stomach discomfort is very bad, or if it affects your appetite, it may be best to be evaluated by your doctor. In

In [8]:
messages = [
    {"role": "user", "content": "My patient woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc! "},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, 
    return_tensors = "pt",
    padding = True, 
).to("cuda")

attention_mask = (inputs != tokenizer.pad_token_id).long()  

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask, 
    streamer=text_streamer,
    max_new_tokens=256,
    use_cache=True,
    temperature=1.0,
    min_p=0.1,
    min_new_tokens=50,
    top_p=0.9
)

Hello, Welcome to Chat Doctor. I read carefully your query and understand your concern. From your symptoms, it seems that you are having some form of motion sickness. If you don't have any other symptoms like fever, vomiting for long time, it is not a concern. It should resolve with time. If you have other symptoms like fever, vomiting for long time, then you should consult a doctor. Hope I have answered your question. If you have any further questions, then you may please ask me. I hope I answered your question. Thank you.<|eot_id|>


In [17]:
model.save_pretrained("Models/HCM_4BIT00006")
tokenizer.save_pretrained("Models/HCM_4BIT00006")
model_name = "model\\Llama-3.2-3B-Instruct-Frog"
finetune_model_name = "Models/HCM_4BIT00006"
device_map = {"": 0}
# Reload and merge
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
modelsave = PeftModel.from_pretrained(base_model, finetune_model_name)
modelsave = modelsave.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

c:\Python\mlperf_env\lib\site-packages\peft\utils\save_and_load.py:195: UserWarning: Could not find a config file in model\Llama-3.2-3B-Instruct-Frog - will assume that the vocabulary was not modified.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.13s/it]


In [18]:
# Directory path where model and tokenizer will be saved
save_dir = "Models/4BIT00006"

# Create directory if it does not exist
os.makedirs(save_dir, exist_ok=True)

# Save model in .bin format with possible sharding
modelsave.save_pretrained(save_dir, safe_serialization=False)
tokenizer.save_pretrained(save_dir)

('Models/4BIT00008\\tokenizer_config.json',
 'Models/4BIT00008\\special_tokens_map.json',
 'Models/4BIT00008\\tokenizer.json')